**Автор:** Сергей Шмаков

**Обновление этого документа** и не только публикуется на канале в Телеграм - https://telegram.me/smm_python

Другие мои площадки:
* ВКонтакте - https://vk.com/smmblog
* Youtube - https://youtube.com/smmblog
* Блог - http://freesmm.ru

# Урок 4: Получить информацию о 100 постах в сообществе ВКонтакте

**запустите код ниже**

In [1]:
from IPython.display import YouTubeVideo
YouTubeVideo("3YDhM037BWU")

**Задача: **
> Получить всю возможную информацию о постах в сообществе

Материалы для изучения:
* Модуль  - http://pythonworld.ru/moduli/modul-time.html
* Модуль  - http://pythonworld.ru/moduli/modul-datetime.html

*Давайте разберёмся с задачей, которая перед нами поставлена.* 

Вся возможная информация это какая? 
Отправляемся в раздел для разработчиков в методы https://vk.com/dev/. Нам нужно что-то про "Стену" - https://vk.com/dev/wall 

* wall.get, Возвращает список записей со стены пользователя или сообщества. - https://vk.com/dev/wall.get
То, что нужно. Теперь разберёмся как он работает



Параметр | Описание | Тип данных
--- | --- | ---
owner_id | идентификатор пользователя или сообщества, со стены которого необходимо получить записи (по умолчанию — текущий пользователь).*Обратите внимание, идентификатор сообщества в параметре owner_id необходимо указывать со знаком "-" — например, owner_id=-1 соответствует идентификатору сообщества ВКонтакте API (club1) | целое число`
domain | короткий адрес пользователя или сообщества. | строка
offset | смещение, необходимое для выборки определенного подмножества записей.  | положительное число
count | количество записей, которое необходимо получить (**но не более 100**). | положительное число
filter | определяет, какие типы записей на стене необходимо получить. Возможны следующие значения параметра: <br>suggests — предложенные записи на стене сообщества (доступно только при вызове с передачей **access_token**);<br>postponed — отложенные записи (доступно только при вызове с передачей access_token);<br>owner — записи на стене от ее владельца;<br>others — записи на стене не от ее владельца;all — все записи на стене (owner + others).<br> * Если параметр не задан, то считается, что он равен all*  | строка
extended | 1 — будут возвращены три массива **wall**, **profiles** и **groups**. По умолчанию дополнительные поля не возвращаются.  | флаг, может принимать значения 1 или 0
fields | список дополнительных полей для профилей и групп, которые необходимо вернуть. См. описание полей объекта user и описание полей объекта group - https://vk.com/dev/fields_groups . Обратите внимание, этот параметр учитывается только при extended=1.  | строка. список слов, разделенных через запятую

**Результат**
После успешного выполнения возвращает список объектов записей на стене. 

Если был задан параметр **extended=1**, возвращает отдельно списки объектов записей на стене в поле **items**, пользователей с дополнительными полями **photo** и **online** в поле **profiles**, и сообществ в поле groups.

Почитали, увидели, теперь давайте попробуем написать код, чтобы отправить запрос и получить информацию о двух записях в группе (первая просто это закреплённый пост). Для примера я буду работать со своим пабликом http://vk.com/smmblog (id = 38369814 )

> **Обязательно** ознакомьтесь с этой информацией https://vk.com/dev/post

**запустите код ниже**

In [ ]:
import requests
r = requests.get('https://api.vk.com/method/wall.get',params={'owner_id':-38369814,'count':2}) #Получим информацию о 2х постах
r.json()

Как видите, информации очень много. Пост или запись может включать себя другие объекты: фото, видео, документы, музыку, другие записи. 

Давайте обновим задачу, так как информации слишком много и для нас она вся неочень интересна.

**Обновление задачи:**
> * получить количество лайков, репостов, комментариев у каждой записи с датой публикации

**Теория**

Для того, чтобы вытащить нужные нам данные нужно научиться доставать их из Словарей и Списков.
Обращение к объекту списка по его индексу - http://pythonworld.ru/osnovy/indeksy-i-srezy.html (у кого с анйглиский лучше - http://www.jamalmoir.com/2016/05/python-sequence-slicing-guide.html )

Приведу несколько простых примеров для понимания:

In [3]:
list= ['Сергей','Василий', 'smmplanner'] #списки всегда в квадратных скобках, а строки в кавычках
#Чтобы достать из списка Василия нужно обратиться к 1 элементу списка. Порядок элементов в списке начинается с нуля "0"
print(list[1]) #Выведит на экран Василий
print(list[1],'работает в',list[2]) #Выведит на экран Василий работает в smmplanner

Василий
Василий работает в smmplanner


Теперь пример для Словаря. 
Подробнее о Словарях - http://pythonworld.ru/tipy-dannyx-v-python/slovari-dict-funkcii-i-metody-slovarej.html

## Задача про склад с мороженным
Давайте представим, что у нас есть 2 вида мороженного: Пломбир и Эскимо. Продаются они с разной ценой и в разной валюте. Нужно вывести на экран Тип мороженного, цену, валюту и Остаток на складе . Для Эскимо посчитать стоимость остатка на складе по курсу доллара 1$ = 65 руб

In [4]:
dict = {'Мороженное':[{'Тип':'Пломбир', 'Цена':256, 'Валюта':'руб.', 'Склад':10},
                      {'Тип':'Эскимо', 'Цена':221, 'Склад':5, 'Валюта':'$'}]}
print(dict['Мороженное']) #обращение по ключу Словаря
#При обращении по этому ключу мы попадаем в список, внутри которого 2 списка, внутри которых Словари :)))
#Теперь можно обратиться по индексу, чтобы получить информацию из первого[0] или второго списка[1]

[{'Цена': 256, 'Склад': 10, 'Тип': 'Пломбир', 'Валюта': 'руб.'}, {'Цена': 221, 'Склад': 5, 'Тип': 'Эскимо', 'Валюта': '$'}]


Давайте попробуем достать нужные нам элементы по их индексу или ключу
**запустите код ниже**

In [5]:
print('Информация о пломбире', dict['Мороженное'][0])
print('Информация об эскимо', dict['Мороженное'][1])

print('Тип мороженного:', dict['Мороженное'][0]['Тип'])
print('Тип мороженного:', dict['Мороженное'][1]['Тип'])

print('Цена:', dict['Мороженное'][0]['Цена'])
print('Цена:', dict['Мороженное'][1]['Цена'])

print('Валюта:', dict['Мороженное'][0]['Валюта'])
print('Валюта:', dict['Мороженное'][1]['Валюта'])

Информация о пломбире {'Цена': 256, 'Склад': 10, 'Тип': 'Пломбир', 'Валюта': 'руб.'}
Информация об эскимо {'Цена': 221, 'Склад': 5, 'Тип': 'Эскимо', 'Валюта': '$'}
Тип мороженного: Пломбир
Тип мороженного: Эскимо
Цена: 256
Цена: 221
Валюта: руб.
Валюта: $


## Циклы в Python
Давайте теперь представим, что у нас на складе много всяких типов мороженного с разными ценами и с разным количеством на складе. Но мы знаем, что у нас 2 типа мороженных.

**Теория**
Для того, чтобы нам не пришлось писать столько принтов мы используем уже знакомый цикл for, которы будет проходить по списку значений, которые генерирует функция **range**. 
Информация для изучения - http://informatics.mccme.ru/mod/book/view.php?id=2534&chapterid=360

**запустите код ниже**

In [6]:
for i in range(10): #попробуйте заменить значения в скобках на (5,10) и на (5,15,2)
    print(i)

0
1
2
3
4
5
6
7
8
9


Обратите внимание на код ниже
Переменные **type_morojenka**,**price** и другие находятся в теле цикла, потому что нам каждый раз нужно присвоить им новые значения согласно новому индексу

**запустите код ниже**

In [10]:
dict = {'Мороженное':[{'Тип':'Пломбир', 'Цена':256, 'Валюта':'руб./шт', 'Склад':10},
                      {'Тип':'Эскимо', 'Цена':221, 'Склад':5, 'Валюта':'$/шт'}]}
sklad_morojenki = dict['Мороженное']

for i in range(2): #вместо 2 мы могли подставить значение len(dict['Мороженное'])
                   #считает сколько элементов в списке или чему равна его длина
    type_morojenka = dict['Мороженное'][i]['Тип']
    price = dict['Мороженное'][i]['Цена']
    cy = dict['Мороженное'][i]['Валюта']
    value = dict['Мороженное'][i]['Склад']
    print(type_morojenka,',','Стоимость:',price, cy,'Количество на складе',value)
    if type_morojenka == 'Эскимо':
        print('Стоимость остатков Эскимо на складе:' + str(value*65) +' руб.')

Пломбир , Стоимость: 256 руб./шт Количество на складе 10
Эскимо , Стоимость: 221 $/шт Количество на складе 5
Стоимость остатков Эскимо на складе:325 руб.


## Сбор информации о постах в паблике ВКонтакте
Вернёмся к нашим баранам с получением информации о постах в группе. Напомню, что нам нужно получить:
* количество лайков
* количество репостов 
* количество комментариев 
* дату публикацией этих постов
* желательно оформить всё в видео удобной таблицы, либо так, чтобы это можно было скопировать в эксель

Давайте достать дату и время для 2х наших постов, которые мы получили выше. Ниже мы используем тот же самый метод что и в задаче с мороженным

In [11]:
import requests
r = requests.get('https://api.vk.com/method/wall.get',params={'owner_id':-38369814,'count':2})
response = r.json()
post_date1 = response['response'][1]['date'] #выведит первую запись на стене или закреплённый пост
post_date = response['response'][2]['date'] #выведит на экран вторую запись на стене
print(post_date, post_date1)

1462618985 1462001084


Почему дата выглядит так мы рассмотрим в 5 уроке. Сейчас нас это мало заботит.

Как и в задаче с мороженным мы предполагаем, что у нас будет сотри и тысячи публикаций, для которых нужно будет получать информацию. Следовательно, мы не можем для каждой даты создавать переменную таким образом
```python
post_date1 = response['response'][1]['date'] #выведит первую запись на стене или закреплённый пост
post_date = response['response'][2]['date'] #выведит на экран вторую запись на стене
```
и значит, будем использовать цикл **for**

**запустите код ниже**

In [12]:
import requests
r = requests.get('https://api.vk.com/method/wall.get',params={'owner_id':-38369814,'count':2})
response = r.json()
print('Количество постов на стене = ',response['response'][0]) #выводит 1 раз
#Создаём цикл с переменными для каждого поста
for i in range(1,3):
    post_date = response['response'][i]['date'] #дата и время публикации в Unix-формате
    comments_count = response['response'][i]['comments']['count'] #количество комментариев
    post_id = response['response'][i]['id'] #id поста в сообществе
    group_id = response['response'][i]['to_id'] #id сообщества . В версиях API ниже 5.7 вместо поля owner_id приходит to_id.
    likes_count = response['response'][i]['likes']['count'] #количество лайков
    repost_count = response['response'][i]['reposts']['count']  #количество репостов
    signer_id = response['response'][i]['signer_id'] #если есть подпись автора    
    print(post_date,comments_count)

Количество постов на стене =  2365
1462001084 13
1462618985 5


Так, мы смогли вывести на экран количество постов, дату публикации и количество комментариев функцией
```python
print(post_date,comments_count)
```
Теперь нам нужно её доработатать, чтобы информация была понятна для нас с вами. "Странную" дату пока не трогаем. Работает с тем что есть.

В итоге код, чтобы получить информацию о 2х последних постах

**запустите код ниже**

In [13]:
import requests
r = requests.get('https://api.vk.com/method/wall.get',params={'owner_id':-38369814,'count':2})
response = r.json()
print('Количество постов на стене = ',response['response'][0]) #выводит 1 раз
#Создаём цикл с переменными для каждого поста

print('Дата публикации',',','Ссылка',',','Количество лайков',',','Количество репостов',',','Количество комментариев')
for i in range(1,3):
    post_date = response['response'][i]['date'] #дата и время публикации в Unix-формате
    comments_count = response['response'][i]['comments']['count'] #количество комментариев
    post_id = response['response'][i]['id'] #id поста в сообществе
    group_id = response['response'][i]['to_id'] #id сообщества . В версиях API ниже 5.7 вместо поля owner_id приходит to_id.
    likes_count = response['response'][i]['likes']['count'] #количество лайков
    repost_count = response['response'][i]['reposts']['count']  #количество репостов
    print(post_date,',','https://vk.com/wall'+str(group_id)+'_'+str(post_id),',',
          likes_count,',',repost_count,',',comments_count)

Количество постов на стене =  2365
Дата публикации , Ссылка , Количество лайков , Количество репостов , Количество комментариев
1462001084 , https://vk.com/wall-38369814_19543 , 105 , 41 , 13
1462618985 , https://vk.com/wall-38369814_19598 , 22 , 3 , 5


Теперь, чтобы получить 100 постов вам нужно заменить параметр 'counte':100
```python
r = requests.get('https://api.vk.com/method/wall.get',params={'owner_id':-38369814,'count':100})
```

**запустите код ниже**

In [14]:
import requests
r = requests.get('https://api.vk.com/method/wall.get',params={'owner_id':-38369814,'count':100})
response = r.json()
print('Количество постов на стене = ',response['response'][0]) #выводит 1 раз
#Создаём цикл с переменными для каждого поста

print('Дата публикации',',','Ссылка',',','Количество лайков',',','Количество репостов',',','Количество комментариев')
for i in range(1,3):
    post_date = response['response'][i]['date'] #дата и время публикации в Unix-формате
    comments_count = response['response'][i]['comments']['count'] #количество комментариев
    post_id = response['response'][i]['id'] #id поста в сообществе
    group_id = response['response'][i]['to_id'] #id сообщества . В версиях API ниже 5.7 вместо поля owner_id приходит to_id.
    likes_count = response['response'][i]['likes']['count'] #количество лайков
    repost_count = response['response'][i]['reposts']['count']  #количество репостов
    print(post_date,',','https://vk.com/wall'+str(group_id)+'_'+str(post_id),',',
          likes_count,',',repost_count,',',comments_count)

Количество постов на стене =  2365
Дата публикации , Ссылка , Количество лайков , Количество репостов , Количество комментариев
1462001084 , https://vk.com/wall-38369814_19543 , 105 , 41 , 13
1462618985 , https://vk.com/wall-38369814_19598 , 22 , 3 , 5


## Домашнее задание по уроку 4
> 1. Замените значение в цикле с (1,3) на (1, 101) - так вы получите все 100 постов
2. Ознакомьтесь внимательно с методом  https://vk.com/dev/wall.get и разберитесь, что нужно поправить, чтобы получить посты из любого открытого профиля пользователя ВКонтакте

Ответы и вопросы по домашке публикуйте здесь - https://vk.com/topic-38369814_33529150